<a href="https://colab.research.google.com/github/namphung134/Multi_llm_chatbot/blob/main/Web%2Byoutube_crawl%2Bsummary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install requirement libraries**

In [ ]:
!pip install obsei
!pip install transformers
!pip install dateparser
!pip install sentencepiece
!pip install trafilatura
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.8/263.8 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstalling python-dateutil-2.8.2:
      Successfully uninstalled python-dateutil-2.8.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=78ea22dee92caa0bbe7875fdd7b36081fedce5159d9a67ff30c3d815c86d5090
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


# Config google news

In [ ]:
import dateparser
from gnews import GNews
from pydantic import PrivateAttr
from datetime import datetime, date, timedelta, time, timezone
from typing import Any, Dict, List, Optional
from urllib import parse
from obsei.payload import TextPayload
from obsei.misc.utils import DATETIME_STRING_PATTERN, convert_utc_time
from obsei.source.base_source import BaseSource, BaseSourceConfig
from obsei.source.website_crawler_source import (
    BaseCrawlerConfig,
    TrafilaturaCrawlerConfig,
)

GOOGLE_DATE_TIME_QUERY_PATTERN = "%Y-%m-%d"

class GoogleNewsConfig(BaseSourceConfig):
    _google_news_client: GNews = PrivateAttr()
    TYPE: str = "GoogleNews"
    query: str
    country: Optional[str] = "US"
    language: Optional[str] = "en"
    max_results: Optional[int] = 100
    lookup_period: Optional[str] = None
    after_date: Optional[str] = None
    before_date: Optional[str] = None
    fetch_article: Optional[bool] = True
    crawler_config: Optional[BaseCrawlerConfig] = None

    def __init__(self, **data: Any):
        super().__init__(**data)

        if self.lookup_period and self.after_date:
            raise AttributeError("Can't use `lookup_period` and `after_date` both")
        elif not self.after_date and self.before_date:
            raise AttributeError("Can't use `before_date` without `after_date` or `lookup_period`")

        if self.lookup_period:
            after_time = convert_utc_time(self.lookup_period)
            self.after_date = after_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

        if not self.before_date:
            before_time = datetime.combine(date.today(), time(tzinfo=timezone.utc)) + timedelta(days=1)
            self.before_date = before_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

        self._google_news_client = GNews(
            language=self.language,
            country=self.country,
            max_results=self.max_results,
        )

        if not self.crawler_config:
            self.crawler_config = TrafilaturaCrawlerConfig(urls=[])

    def get_client(self) -> GNews:
        return self._google_news_client


class GoogleNewsSource(BaseSource):
    NAME: Optional[str] = "GoogleNews"

    def lookup(self, config: GoogleNewsConfig, **kwargs) -> List[TextPayload]:  # type: ignore[override]
        source_responses: List[TextPayload] = []

        # Get data from state
        id: str = kwargs.get("id", None)
        state: Optional[Dict[str, Any]] = (
            None
            if id is None or self.store is None
            else self.store.get_source_state(id)
        )
        update_state: bool = True if id else False
        state = state or dict()
        lookup_period: str = state.get("since_time", None)
        since_time = None if not lookup_period else convert_utc_time(lookup_period)
        last_since_time = since_time

        last_after_time = convert_utc_time(config.after_date) if config.after_date else None
        if since_time and last_after_time:
            last_after_time = since_time if since_time > last_after_time else last_since_time
        elif not last_after_time:
            last_after_time = datetime.combine(date.today(), time(tzinfo=timezone.utc))

        before_time = convert_utc_time(config.before_date) if config.after_date else None
        if not before_time or before_time > datetime.combine(date.today(), time(tzinfo=timezone.utc)):
            before_time = datetime.combine(date.today(), time(tzinfo=timezone.utc)) + timedelta(days=1)

        google_news_client = config.get_client()
        more_data_exist = True
        while more_data_exist and before_time > last_after_time:
            after_time = before_time - timedelta(days=1)
            after_date = after_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)
            before_date = before_time.strftime(GOOGLE_DATE_TIME_QUERY_PATTERN)

            new_query = f'{config.query}+after:{after_date}+before:{before_date}'
            query = parse.quote(new_query, errors='ignore')

            before_time = after_time

            articles = google_news_client.get_news(query)

            for article in articles:
                published_date = (
                    None
                    if article["published date"] == ""
                    else dateparser.parse(article["published date"])
                )

                if config.fetch_article and config.crawler_config:
                    extracted_data = config.crawler_config.extract_url(url=article["url"])

                    if extracted_data is not None and extracted_data.get("text", None) is not None:
                        article_text = extracted_data["text"]
                        del extracted_data["text"]
                    else:
                        article_text = ""

                    article["extracted_data"] = extracted_data
                else:
                    article_text = article["description"]

                source_responses.append(
                    TextPayload(
                        processed_text=f"{article['title']}.\n\n {article_text}",
                        meta=vars(article) if hasattr(article, "__dict__") else article,
                        source_name=self.NAME,
                    )
                )

                if config.max_results is not None and len(source_responses) >= config.max_results:
                    source_responses = source_responses[:config.max_results]
                    more_data_exist = False
                    break

                if published_date and since_time and published_date < since_time:
                    more_data_exist = False
                    break
                if last_since_time is None or (
                    published_date and last_since_time < published_date
                ):
                    last_since_time = published_date

            if update_state and last_since_time and self.store is not None:
                state["since_time"] = last_since_time.strftime(DATETIME_STRING_PATTERN)
                self.store.update_source_state(workflow_id=id, state=state)

        return source_responses

# Crawl gg news

In [ ]:
import pandas as pd
query="seagames"
max_results=5
lookup_period="3Y"
language="vi"
country="VN"
userName = []
content = []
rating = []
id = []
title = []
platform = []
extracted_data=[]
source_config = GoogleNewsConfig(
query=query,
max_results=max_results,
lookup_period=lookup_period,
language=language,
country=country,
)
source = GoogleNewsSource()
data = source.lookup(source_config)
#print(data)
for i in range(0, max_results):
    data[i]=dict(data[i])
    #print(data[i])
    id.append(None)
    rating.append(None)
    platform.append('Google News')
    extracted_data.append(data[i]["meta"]["extracted_data"])
    Not_none_values = filter(None.__ne__, extracted_data)
    extracted_data = list(Not_none_values)
    for i in range(len(extracted_data)):
      extracted_data[i]=dict(extracted_data[i])
      print(extracted_data[i])
      userName.append(extracted_data[i]["author"])
      title.append(extracted_data[i]["title"])
      content.append(extracted_data[i]["raw_text"])
data=pd.DataFrame(list(zip(title,userName,content)),columns=["title","author","content"])


ERROR:trafilatura.xml:empty link: None {'target': '/lich-thi-dau/lich-thi-daucau-longsea-games-31-hom-nay-vtv6-vtv5-truc-tiep-n20220517085553236.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/bong-da-viet/ket-quabong-da-sea-games-31ket-qua-bong-dau23-viet-nam-moi-nhat-n20220514171621213.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/sea-games-31/truc-tiep-sea-games-31-hom-nay-215-nu-viet-nam-tranh-hcv-voi-thai-lan-n20220521113223135.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/sea-games-31/truc-tiep-sea-games-145-vtv6-vtv5-dien-kinh-boi-loi-bong-chuyen-esports-wushu-n20220513203805963.htm'}
ERROR:trafilatura.xml:empty link: None {'target': '/truc-tiep-bong-da/vtv6-truc-tiep-bong-da-nu-viet-nam-vs-thai-lan-chung-ket-sea-games-31-19h00-215-n20220520115639702.htm'}


{'title': 'Lịch thi đấu và trực tiếp cầu lông SEA Games 31 (VTV6, VTV5)', 'author': 'PV; BTV-TTVH', 'hostname': 'thethaovanhoa.vn', 'date': '2022-05-19', 'categories': 'Thể thao', 'tags': 'Lich thi dau Sea games 31, lich thi dau sea games, trực tiếp cầu lông Sea games 31, xem trực tiếp cầu lông sea games, lich thi dau cau long, trực tiếp cầu lông việt nam;Lich thi dau Sea games 31;lich thi dau sea games;trực tiếp cầu lông Sea games 31;xem trực tiếp cầu lông sea games;lich thi dau cau long;trực tiếp cầu lông việt nam', 'fingerprint': 'j+u6ya16H0vdxcOe4epzsKjBhXE=', 'id': '91ec24fb', 'license': None, 'comments': None, 'raw_text': 'Lịch thi đấu và trực tiếp cầu lông SEA Games 31 (VTV6, VTV5) Thứ Năm, 19/05/2022 10:55 GMT+7 Lịch thi đấu và trực tiếp cầu lông SEA Games 31 - Xem trực tiếp cầu lông SEA Games 31 năm 2022 tại Việt Nam. Lịch thi đấu môn cầu lông SEA Games 31. Lịch thi đấu cầu lông SEA Games 31 - Xem VTV6 VTV5 trực tiếp cầu lông SEA Games 31 năm 2022. Lịch thi đấu môn cầu lông Vi

In [ ]:
data=data.drop_duplicates()

In [ ]:
data

,title,author,content
0,Đảm bảo mức cao nhất trong công tác y tế phục ...,Lê Hảo TTXVN; Vietnam,Đảm bảo mức cao nhất trong công tác y tế phục ...
2,"IPC không cho phép Nga, Belarus tham gia Paral...",Thúc Anh TTXVN; Vietnam,"IPC không cho phép Nga, Belarus tham gia Paral..."
5,U23 Đông Nam Á 2022: Tuyển Việt Nam bắt đầu hà...,Minh Tiến TTXVN; Vietnam,U23 Đông Nam Á 2022: Tuyển Việt Nam bắt đầu hà...
9,Phát hiện trên 27.000 trường hợp vi phạm nồng ...,Chu Thanh Vân TTXVN; Vietnam,Phát hiện trên 27.000 trường hợp vi phạm nồng ...


# config website

In [ ]:
import json
import logging
from abc import abstractmethod
from typing import List, Optional

import mmh3

from obsei.payload import TextPayload
from obsei.source.base_source import BaseSource, BaseSourceConfig

logger = logging.getLogger(__name__)


class BaseCrawlerConfig(BaseSourceConfig):
    TYPE: str = "BaseCrawler"

    @abstractmethod
    def extract_url(self, url: str, url_id: str = None):
        pass

    @abstractmethod
    def find_urls(self, url: str):
        pass


class TrafilaturaCrawlerConfig(BaseCrawlerConfig):
    # To understand about these configuration params refer:
    # https://trafilatura.readthedocs.io/
    _output_format: str = "json"
    TYPE: str = "Crawler"
    urls: List[str]
    include_comments: bool = False
    include_tables: bool = True
    no_fallback: bool = False
    include_images: bool = False
    include_formatting: bool = False
    deduplicate: bool = True
    no_ssl: bool = False
    is_feed: bool = False
    is_sitemap: bool = False
    include_links: bool = True
    target_language: Optional[str] = None
    url_blacklist: Optional[List[str]] = None

    def extract_url(self, url: str, url_id: str = None):
        try:
            from trafilatura import extract, fetch_url
        except:
            logger.error("Trafilatura is not installed, install as follows: pip install trafilatura")
            return []

        url_id = url_id or "{:02x}".format(mmh3.hash(url, signed=False))
        url_content = fetch_url(
            url=url,
            no_ssl=self.no_ssl,
        )
        extracted_dict = None
        if url_content is not None:
            extracted_data = extract(
                filecontent=url_content,
                record_id=url_id,
                no_fallback=self.no_fallback,
                output_format=self._output_format,
                include_comments=self.include_comments,
                include_tables=self.include_tables,
                include_images=self.include_images,
                include_formatting=self.include_formatting,
                include_links=self.include_links,
                deduplicate=self.deduplicate,
                url_blacklist=self.url_blacklist,
                target_language=self.target_language,
            )

            if extracted_data:
                extracted_dict = json.loads(extracted_data)
                if "raw-text" in extracted_dict:
                    del extracted_dict["raw-text"]

        return extracted_dict

    def find_urls(self, url: str):
        try:
            from trafilatura import feeds, sitemaps
        except:
            logger.error("Trafilatura is not installed, install as follows: pip install trafilatura")
            return []

        urls: List[str] = []
        if self.is_sitemap:
            urls = sitemaps.sitemap_search(url=url, target_lang=self.target_language)
        elif self.is_feed:
            urls = feeds.find_feed_urls(url=url, target_lang=self.target_language)

        return urls


class TrafilaturaCrawlerSource(BaseSource):
    NAME: Optional[str] = "Crawler"

    def lookup(  # type: ignore[override]
        self, config: TrafilaturaCrawlerConfig, **kwargs
    ) -> List[TextPayload]:
        source_responses: List[TextPayload] = []

        final_urls = []
        if config.is_sitemap or config.is_feed:
            for url in config.urls:
                final_urls.extend(config.find_urls(url=url))
        else:
            final_urls = config.urls

        for url in final_urls:
            extracted_data = config.extract_url(url=url)
            if extracted_data is None:
                logger.warning(f"Unable to crawl {url}, hence skipping it")
                continue
            comments = (
                "" if "comments" not in extracted_data else extracted_data["comments"]
            )
            source_responses.append(
                TextPayload(
                    processed_text=f"{extracted_data['text']}. {comments}",
                    meta=extracted_data,
                    source_name=self.NAME,
                )
            )

        return source_responses

By default `pip install obsei` will only install core dependencies.
To install all required dependencies use `pip install obsei[all]`.
Refer https://obsei.com/#install-obsei for more options.



In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/data_VH/lop5_data_needcrawl.xlsx')

In [ ]:
df

,tác phẩm,link
0,thư gửi các em học sinh,https://baivan.net/content/giai-bai-tap-doc-th...
1,quang cảnh làng mạc ngày mưa,https://baivan.net/content/giai-bai-tap-doc-qu...
2,nghìn năm văn hiến,https://baivan.net/content/giai-bai-tap-doc-ng...
3,màu sắc em yêu,https://baivan.net/content/giai-bai-tap-doc-ma...
4,lòng dân 1,https://baivan.net/content/giai-bai-tap-doc-lo...
5,lòng dân 2,https://baivan.net/content/giai-bai-tap-doc-lo...
6,những con sếu bằng giấy,https://baivan.net/content/giai-bai-tap-doc-nh...
7,bài ca về trái dất,https://baivan.net/content/giai-bai-tap-doc-ba...
8,một chuyên gia máy xúc,https://baivan.net/content/giai-bai-tap-doc-mo...
9,"ê-mi-li, con…",https://baivan.net/content/giai-bai-tap-doc-e-...


In [ ]:
urls = df['link'].tolist()

# Crawl website

In [ ]:
# from obsei.source.website_crawler_source import (
#     TrafilaturaCrawlerConfig,
#     TrafilaturaCrawlerSource,
# )
import pandas as pd
#Single URL
from trafilatura import extract, fetch_url
source_config=TrafilaturaCrawlerConfig(
    urls=["https://cas-solution.vn/crawl/"],
    include_comments = False,
    # include_tables = False,
    # no_fallback = False,
    # include_images = False,
    # include_formatting = False,
    # deduplicate = False,
    # no_ssl = False,
    # is_feed = False,
    #is_sitemap = True,
    # include_links = False,
)
source=TrafilaturaCrawlerSource()
urls=["https://cas-solution.vn/crawl/"]
source_response_list=source.lookup(source_config)
print(source_response_list)
#Extract information from data
id = []
hostname = []
title = []
raw_text = []
rating = []
platform = []
for i in range(0, len(urls)):
    print(i)
    source_response_list[i] = dict(source_response_list[i])
    print(source_response_list)
    # id.append(source_response_list[i]["meta"]["id"])
    # hostname.append(source_response_list[i]["meta"]["hostname"])
    # title.append(source_response_list[i]["meta"]["title"])
    # raw_text.append(source_response_list[i]["meta"]["raw_text"])
data = pd.DataFrame(list(zip(id, hostname, title, raw_text)), columns=['id','author','title','content'])

[TextPayload(segmented_data={}, meta={'text': '[Crawl là gì](https://cas-solution.vn/crawl/)? Đây là câu hỏi của nhiều Marketer khi mới bắt đầu tìm hiểu về SEO hay phát triển nội dung trên website. Vậy Crawl website hoạt động như nào? Công cụ này đóng vai trò gì trong việc gia tăng thứ hạng, đánh giá điểm chất lượng trên website? Hãy cùng CAS SOLUTION tìm hiểu ngay trong bài viết dưới đây nhé!\nCrawl là gì?\nCrawl là gì? Google Crawl (searchbot, spider hay bộ thu thập dữ liệu) là một phần mềm mà Google và các công cụ tìm kiếm khác sử dụng để quét trang Web. Nói một cách đơn giản, nó “thu thập” dữ liệu web từ trang này sang trang khác, tìm kiếm nội dung mới hoặc nội dung cập nhật mà Google chưa có trong cơ sở dữ liệu của mình.\nSong, bất kỳ công cụ tìm kiếm nào cũng có bộ trình thu thập thông tin riêng. Đối với Google, có hơn 15 loại trình thu thập thông tin khác nhau và trình thu thập thông tin chính của Google được gọi là Googlebot. Googlebot thực hiện cả thu thập thông tin và lập chỉ

In [ ]:
data["content"]

,content


In [ ]:
import re
for val in data["content"].values:
  text=str(val)
  print(text)

#Apply Model to summary

In [ ]:
import torch
if torch.cuda.is_available():
   device = torch.device("cuda")
else:
   device = torch.device("cpu")

In [ ]:
from langdetect import detect
for index, row in data.iterrows():
    text = row['content']
    if detect(text) == "vi":
        import pandas as pd
        from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
        import torch
        tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
        model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        preprocess_text = text.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
           tokenized_text,
           max_length=512,
           num_beams=4,
           repetition_penalty=2.5,
           length_penalty=1.0,
           early_stopping=True
    )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("\n\nSummarized: \n", output)

TypeError: ignored

In [ ]:
from langdetect import detect
for index, row in data.iterrows():
    text = row['content']
    if detect(text) == "vi":
        import pandas as pd
        from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
        import torch
        tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
        model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        preprocess_text = text1.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
           tokenized_text,
           max_length=512,
           num_beams=4,
           repetition_penalty=2.5,
           length_penalty=1.0,
           early_stopping=True
    )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        print("\n\nSummarized: \n", output)



Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


Summarized: 
 Công ty Cổ phần Vận tải đường sắt Hà Nội cho biết, sau thời gian dài đình trệ do Covid-19, doanh nghiệp sẽ chạy lại đôi tàu khách SP3/SP4 phục vụ khách du lịch Sa Pa dịp cuối tuần, nếu khách đông sẽ cho chạy tàu hàng ngày.


# test

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")

In [ ]:
def summarization(data,tokenizer,model):
    from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
    import torch
    from langdetect import detect
    import pandas as pd
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
    for val in data["content"].values:
        text=str(val)
    for index, row in data.iterrows():
        text = row['content']
        if detect(text) == "vi":
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model = model.to(device)
            preprocess_text = text.strip().replace("\n", "")
            tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
            summary_ids = model.generate(
                tokenized_text,
                max_length=512,
                num_beams=8,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )
            output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
            print("\n\nSummarized: \n", output)
            data.loc[index, 'summary'] = output
    return data

In [ ]:
summarization(data=data,tokenizer=tokenizer,model=model)

TypeError: ignored

In [ ]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
tokenizer.save_pretrained
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = model.to(device)
def summarization(text):
        preprocess_text = text.strip().replace("\n", "")
        tokenized_text = tokenizer.encode(preprocess_text, return_tensors="pt").to(device)
        summary_ids = model.generate(
                tokenized_text,
                max_length=256,
                num_beams=4,
                repetition_penalty=2.5,
                length_penalty=1.0,
                early_stopping=True
            )
        output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return output

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
import pandas as pd
tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
tokenizer.save_pretrained('/content/drive/MyDrive/summary/tokenizer_pretrained')
model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
model.save_pretrained('/content/drive/MyDrive/summary/model_pretrained')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
summarization(text="""Địa phương sẽ chi 670 tỷ đồng, trong tổng số 1.333 tỷ đồng giải phóng mặt bằng đoạn 19,5 km cao tốc Biên Hòa – Vũng Tàu qua địa phận tỉnh này.
Nội dung được thông qua tại kỳ họp HĐNĐ tỉnh Bà Rịa – Vũng Tàu khóa VII, nhiệm kỳ 2021-2026, ngày 12/4.
Động thái của tỉnh đưa ra sau khi dự án tuyến đường được Thủ tướng chấp thuận đầu tư bằng ngân sách nhà nước.
Cao tốc Biên Hòa – Vũng Tàu (giai đoạn 1) dài 53,7 km, quy mô 4-6 làn xe, tổng mức đầu tư 17.837 tỷ đồng, dự kiến khởi công đầu năm sau và hoàn thành năm 2025.""")

'Tỉnh Bà Rịa – Vũng Tàu sẽ chi 670 tỷ đồng, trong tổng số 1.333 tỷ đồng giải phóng mặt bằng đoạn 19,5 km cao tốc Biên Hòa – Vũng Tàu qua địa phận tỉnh này. Động thái của tỉnh đưa ra sau khi dự án tuyến đường được Thủ tướng chấp thuận đầu tư bằng ngân sách nhà nước.'

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00


In [ ]:
from selenium import webdriver

def crawl_questions_answers(url):
    # Khởi tạo trình duyệt
    driver = webdriver.Chrome()  # Chọn trình duyệt của bạn (Chrome, Firefox, ...)
    driver.get(url)

    # Đợi cho trang web tải hoàn thành (có thể cần tùy chỉnh thời gian chờ)
    # Thông qua implicit wait
    driver.implicitly_wait(10)

    # Lấy danh sách các câu hỏi và câu trả lời
    questions = driver.find_elements_by_xpath('//div[@class="entry-content"]/strong')
    answers = driver.find_elements_by_xpath('//div[@class="entry-content"]/p')

    # Lưu kết quả vào một list
    result = []
    for question, answer in zip(questions, answers):
        result.append({
            "question": question.text,
            "answer": answer.text
        })

    # Đóng trình duyệt
    driver.quit()

    return result

# Sử dụng hàm để crawl dữ liệu
url = "https://baivan.net/content/giai-bai-tap-doc-thu-gui-cac-hoc-sinh.html"
data = crawl_questions_answers(url)

# In ra kết quả
for entry in data:
    print(f"Question: {entry['question']}")
    print(f"Answer: {entry['answer']}")
    print("="*30)


SessionNotCreatedException: ignored